# 08-01 Zod 数据验证

Zod 是 TypeScript 优先的 Schema 验证库，OpenClaw 使用 Zod 验证配置和数据。

## 1. 基础类型

In [ ]:
import { z } from 'zod';

// 字符串
const nameSchema = z.string();
nameSchema.parse('Alice');  // ✓
// nameSchema.parse(123);   // ✗ 抛出错误

// 数字
const ageSchema = z.number().min(0).max(150);
ageSchema.parse(25);  // ✓

// 布尔值
const flagSchema = z.boolean();

// 可选值
const optionalSchema = z.string().optional();
optionalSchema.parse(undefined);  // ✓
optionalSchema.parse('hello');    // ✓

## 2. 对象 Schema

In [ ]:
// 用户 Schema
const UserSchema = z.object({
  id: z.number(),
  name: z.string().min(2),
  email: z.string().email(),
  age: z.number().optional(),
  role: z.enum(['user', 'admin']).default('user')
});

// 类型推断
type User = z.infer<typeof UserSchema>;
// type User = {
//   id: number;
//   name: string;
//   email: string;
//   age?: number;
//   role: 'user' | 'admin';
// }

// 验证
const user = UserSchema.parse({
  id: 1,
  name: 'Alice',
  email: 'alice@example.com'
});

## 3. 安全解析

In [ ]:
// 使用 safeParse 不抛出异常
const result = UserSchema.safeParse({
  id: 1,
  name: 'A',  // 太短
  email: 'invalid-email'  // 格式错误
});

if (!result.success) {
  console.log('验证失败:', result.error.format());
  // {
  //   name: { _errors: ['String must contain at least 2 character(s)'] },
  //   email: { _errors: ['Invalid email'] }
  // }
}

## 4. 高级用法

In [ ]:
// 数组
const TagsSchema = z.array(z.string()).max(5);

// 联合类型
const StatusSchema = z.union([
  z.literal('pending'),
  z.literal('success'),
  z.literal('error')
]);

// 自定义验证
const PasswordSchema = z.string()
  .min(8)
  .refine((val) => /[A-Z]/.test(val), {
    message: '必须包含大写字母'
  })
  .refine((val) => /[0-9]/.test(val), {
    message: '必须包含数字'
  });

// 转换
const NumberFromString = z.string().transform((val) => parseInt(val, 10));
NumberFromString.parse('42');  // 42 (number)

## OpenClaw 实际案例

In [ ]:
// OpenClaw 配置验证示例
const ChannelConfigSchema = z.object({
  type: z.enum(['telegram', 'discord', 'slack']),
  enabled: z.boolean().default(true),
  token: z.string().min(1),
  webhookUrl: z.string().url().optional(),
  rateLimit: z.object({
    maxRequests: z.number().positive(),
    windowMs: z.number().positive()
  }).optional()
});

type ChannelConfig = z.infer<typeof ChannelConfigSchema>;

// 使用
const config = ChannelConfigSchema.parse({
  type: 'telegram',
  token: 'bot123456:ABC-DEF1234',
  rateLimit: {
    maxRequests: 100,
    windowMs: 60000
  }
});

## 练习

1. 为 API 请求设计验证 Schema
2. 实现表单验证（用户名、密码、邮箱）
3. 查看 OpenClaw 源码中的 zod 使用